In [1]:
# packages and plot parameters
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import TwoSlopeNorm
import numpy.ma as ma
import glob
import os
import re
from operator import mul
import sys

plt.rcParams['figure.figsize'] = (10,4)

sys.path.append("/home/mmurakami/MITgcm/MITgcm_c68r/MITgcm-checkpoint68r/utils/python/MITgcmutils/MITgcmutils/") # go to parent dir
from mds import *

# add rdmds reading functions to path
sys.path.append("/home/mmurakami/jupyterfiles/") # go to parent dir
from read_binary import *

# add personal functions to path test
from calc_UV_conv_1face import calc_UV_conv_1face
from calc_mskmean_T_mod import calc_mskmean_T_mod
from mk3D_mod import mk3D_mod

### Load some data

In [2]:
nx=20
ny=16
nz=23

In [3]:
# read in the files - she did .data but I guess I will do .meta
dirrun = "/scratch2/atnguyen/labsea/layers/run_c68r_layers_03Jun2023_noOL_10d/"
dirIn = dirrun + "diags/"
dirGrid = "/scratch2/atnguyen/labsea/GRID/"
flist = rdmds(dirIn + "TRSP/trsp_3d_set1", np.nan)

In [89]:
mygrid = {
    'dirGrid': dirGrid,
    'nFaces': 1,
    'fileFormat': 'compact',
    'memoryLimit': 2,
    'ioSize': [nx*ny, 1],
    'facesSize': [ny, nx],
    'facesExpand': [ny, nx],
    'missVal': 0,
}

fldstr2d = ['XC','YC','XG','YG','RAC','Depth','DXG','DYG','DXC','DYC']
fldstr3d = ['hFacC','hFacW','hFacS','mskC','mskS','mskW']
fldstr3dp = ['hFacC','hFacW','hFacS','maskCtrlC','maskCtrlS','maskCtrlW']
fldstr1d = ['RC','RF','DRC','DRF']

for fld in fldstr1d:
    mygrid[fld] = np.squeeze(rdmds(os.path.join(dirGrid, fld)))

for fld in fldstr3d:
    temp = rdmds(os.path.join(dirGrid, fldstr3dp[fldstr3d.index(fld)]))
    mygrid[fld] = temp.reshape(nz, ny, nx)

for fld in fldstr2d:
    temp = rdmds(os.path.join(dirGrid, fld))
    mygrid[fld] = temp.reshape(ny, nx)

areaW, areaS, Vol = [], [], []
for k in range(nz):
    areaW.append(mygrid['DYG'] * mygrid['DRF'][k])
    areaS.append(mygrid['DXG'] * mygrid['DRF'][k])
    Vol.append(mygrid['RAC'] * mygrid['DRF'][k])

dxg = mygrid['DXG']
drf = mygrid['DRF']
dyg = mygrid['DYG']
rac = mygrid['RAC']
hfacC = mygrid['hFacC']

In [90]:
# time step 2
u = flist[1][0]
v = flist[1][1]

# tile these
utrp = u * np.tile(dyg,(nz,1,1)) * np.tile(np.reshape(drf,(23,1,1)), (1,16,20))
vtrp = v * np.tile(dyg,(nz,1,1)) * np.tile(np.reshape(drf,(23,1,1)), (1,16,20))

### Single face convergence using calc_UV_conv_1face

#### /home/mmurakami/matlab/lookat_labsea.m

In [91]:
# do the same manually - this function works!
ix = 11
jy = 9
kz = 0

fldOut = calc_UV_conv_1face(utrp,vtrp)

fldOut[kz,jy,ix] - (-(utrp[kz,jy,ix+1] - utrp[kz,jy,ix]) - (vtrp[kz,jy+1,ix] - vtrp[kz,jy,ix]))

0.0

### Masked mean with calc_mskmean_T_mod

In [92]:
# set the variables for the input of mskmean
# utrp, vtrp
a = {}
a['utrp'] = utrp
a['vtrp'] = vtrp
msk = np.zeros((ny,nx))
msk[12,5] = 2
msk.shape

(16, 20)

In [93]:
b, area = calc_mskmean_T_mod(a,msk,rac,"extensive")

In [94]:
b

{'utrp': -5.52567770836106e-07, 'vtrp': 0.0}

In [95]:
tmp.shape

(10, 18, 16, 20)

In [96]:
# another test
tmp = rdmds(dirIn + "BUDG/exf_zflux_set1", np.nan)
var_names = ['EXFpreci', 'EXFevap', 'EXFroff', 'EXFempmr', 'EXFswdn', 'EXFlwdn', 'EXFswnet',
             'EXFlwnet', 'EXFqnet', 'EXFatemp', 'EXFaqh', 'EXFtaux', 'EXFtauy', 'EXFuwind',
             'EXFvwind', 'EXFpress', 'EXFhs', 'EXFhl']

# Create a dictionary with variable names and corresponding data
a = {var_names[i]: tmp[i,:, :] for i in range(tmp.shape[0])}


msk = np.zeros((ny, nx))
msk[11:14, 8:12] = 1
msk = msk * hfacC[0,:, :]

mskg = np.ones((ny, nx)) * hfacC[0, :, :]

In [97]:
def calc_mskmean_T_mod(fldIn, mask, RAC, fldType="intensive"):
    import numpy as np
    
    # If fldIn is a dictionary - run the function again
    if isinstance(fldIn, dict):
        fldOut = {}
        list0 = fldIn.keys()
        for key, value in fldIn.items():
            if isinstance(value, (float, int, np.ndarray)):
                tmp2, area = calc_mskmean_T_mod(value, mask, RAC, fldType)
                fldOut[key] = tmp2
        return fldOut, area

    # if it is not a dictionary, continue
    nr = fldIn.shape[0]
    nr2 = mask.shape[0]
    
    if nr2 != nr:
        mask = np.tile(mask, (nr, 1, 1))

    #mask[mask == 0] = np.nan
    #mask[fldIn == np.nan] = np.nan
    #mask[np.isnan(fldIn)] = np.nan
    
    areaMask = np.tile(RAC, (nr, 1, 1)) * mask
    
    if fldType == "intensive":
        #fldOut = np.nansum(fldIn * areaMask) / np.nansum(areaMask)
        fldOut = np.sum(np.sum(fldIn * areaMask)) / np.sum(np.sum(areaMask))
        area = np.nansum(areaMask)
    else:
        #fldOut = np.nansum(fldIn * mask) / np.nansum(areaMask)
        fldOut = np.sum(np.sum(fldIn * mask)) / np.sum(np.sum(areaMask))
        area = np.nansum(areaMask)
        
    return fldOut, area

In [101]:
b,area = calc_mskmean_T_mod(a, msk, rac, 'intensive')
bg,area = calc_mskmean_T_mod(a, mskg, rac, 'intensive')

# Perform calculations for intensive
result1 = np.sum(np.sum(a['EXFpreci'] * rac2 * hfacC[0,:, :] * msk)) / np.sum(np.sum(rac2 * hfacC[0,:, :] * msk)) - b['EXFpreci']
result2 = np.sum(np.sum(a['EXFpreci'] * rac2 * hfacC[0,:, :])) / np.sum(np.sum(rac2 * hfacC[0,:, :])) - bg['EXFpreci']
print(result1,result2)

b,area = calc_mskmean_T_mod(a, msk, rac, 'extensive')
bg,area = calc_mskmean_T_mod(a, mskg, rac, 'extensive')

# perform calculations for extensive
result1 = np.sum(np.sum(a['EXFpreci'] * msk)) / np.sum(np.sum(rac2 * hfacC[0,:, :] * msk)) - b['EXFpreci']
result2 = np.sum(np.sum(a['EXFpreci'] * rac2 * hfacC[0,:, :])) / np.sum(np.sum(rac2 * hfacC[0,:, :])) - bg['EXFpreci']
print(result1)

0.0 0.0
0.0


In [84]:
b,area = calc_mskmean_T_mod(a['EXFpreci'],msk,rac,'intensive')
b

5663.441564898244

In [68]:
msk.shape

(16, 20)

In [81]:
rac2 = np.tile(rac, (18, 1, 1))
msk2 = np.tile(msk, (18,1,1))
np.sum(np.sum(a['EXFpreci'] * rac2 * hfacC[0,:, :] * msk)) / np.sum(np.sum(rac2 * hfacC[0,:, :] * msk))

5663.441564898244

In [74]:
areaMask = np.tile(rac, (18, 1, 1)) * msk2
np.sum(np.sum(a['EXFpreci'] * areaMask)) / np.sum(np.sum(areaMask))

5663.441564898244

In [63]:
b['EXFpreci']

3.5192097350144385e-07

### mk3D mod

In [13]:
# read in the files - she did .data but I guess I will do .meta
dirrun = "/scratch2/atnguyen/labsea/layers/run_c68r_layers_03Jun2023_noOL_10d/"
dirIn = dirrun + "diags/BUDG/"
dirGrid = "/scratch2/atnguyen/labsea/GRID/"

# read all time stepts for one example snap data - there are 10
flist = rdmds(dirIn + "budg2d_snap_set1", np.nan)
# this is shape 10,9,16,20
# 10 - time steps as files/days
# 9 - fldList = {'ETAN    ' 'SIheff  ' 'SIhsnow ' 'SIarea  ' 'sIceLoad' 'PHIBOT  ' 'ETANa   ' 'ETANb   ' 'ETANc   ' };

# get time-steps:
flist = [f for f in os.listdir(dirIn) if f.startswith('budg2d_snap_set1.') and f.endswith('.data')]
idot = flist[0].index('.')
idot = [idot+1, flist[0][idot+1:].index('.')+idot+1]
idot = np.asarray(idot,dtype=int)

mygrid = {
    'dirGrid': dirGrid,
    'nFaces': 1,
    'fileFormat': 'compact',
    'memoryLimit': 2,
    'ioSize': [nx*ny, 1],
    'facesSize': [ny, nx],
    'facesExpand': [ny, nx],
    'missVal': 0,
}

fldstr2d = ['XC','YC','XG','YG','RAC','Depth','DXG','DYG','DXC','DYC']
fldstr3d = ['hFacC','hFacW','hFacS','mskC','mskS','mskW']
fldstr3dp = ['hFacC','hFacW','hFacS','maskCtrlC','maskCtrlS','maskCtrlW']
fldstr1d = ['RC','RF','DRC','DRF']

for fld in fldstr1d:
    mygrid[fld] = np.squeeze(rdmds(os.path.join(dirGrid, fld)))

for fld in fldstr3d:
    temp = rdmds(os.path.join(dirGrid, fldstr3dp[fldstr3d.index(fld)]))
    mygrid[fld] = temp.reshape(nz, ny, nx)

for fld in fldstr2d:
    temp = rdmds(os.path.join(dirGrid, fld))
    mygrid[fld] = temp.reshape(ny, nx)

mygrid['mskC'][mygrid['mskC'] == 0] = np.nan

areaW, areaS, Vol = [], [], []
for k in range(nz):
    areaW.append(mygrid['DYG'] * mygrid['DRF'][k])
    areaS.append(mygrid['DXG'] * mygrid['DRF'][k])
    Vol.append(mygrid['RAC'] * mygrid['DRF'][k])

RAC = mygrid['RAC']

In [33]:
# path to this is /workspace/atnguyen/atn_tools/gcmfaces_mod on nansen
def mk3D_mod(arr_lessd, arr_3d):
    '''
    inputs
        arr_lessd - the array we want to make 3D
        arr_3d - the array in the shape we want
    outputs
        a - the modified array 
    '''
    import numpy as np
    # we will skip out on An's other definitions for gcmfaces and cells
    # Check if c is a double (numpy array)
    if isinstance(arr_3d, np.ndarray):
        nz = arr_3d.shape[0]
        full_size = np.array(arr_3d.shape)
        half_size = np.array(arr_lessd.shape)
        
        # If conditions for 2D->3D
        # go from 2D field to 3D field
        if len(half_size) == 2:
            tmp1 = arr_lessd.copy()
            n1 = arr_lessd.shape[0]
            n2 = arr_lessd.shape[1]
            #tmp1 = tmp1.flatten()
            #tmp1 = np.dot(arr_lessd.reshape(-1, 1), np.ones((1, arr_3d.shape[0])))
            #tmp1 = tmp1.reshape(arr_3d.shape[0],n1,n2)
            tmp1 = tmp1[np.newaxis,:,:] * np.ones((arr_3d.shape[0],1,1))
            a = tmp1

        # If conditions for 1D->3D
        elif len(half_size) == 1:
            tmp1 = arr_3d.copy()
            tmp2 = tmp1.shape
            n1 = tmp2[2]
            n2 = tmp2[1]
            #tmp1 = np.dot(np.ones((n1*n2,1)),arr_lessd[np.newaxis,:])
            #tmp1 = np.reshape(tmp1,(arr_3d.shape[0],n2,n1))
            tmp1 = np.ones((1,n2,n1)) * arr_lessd[:,np.newaxis,np.newaxis]
            a = tmp1
    return a

In [34]:
tmptend = np.zeros((nz, ny, nx))
hfC = mygrid['hFacC']
hfC[hfC == 0] = np.nan
DD = mygrid['Depth']
file_name = 'budg2d_snap_set1'
budg2d_snap_set1 = rdmds(os.path.join(dirIn, file_name),np.nan)
budg2d_snap_set1 = budg2d_snap_set1[:2]  # only first two time steps
ETAN = budg2d_snap_set1[:,0,:,:]
ETAN =  (ETAN[1, :, :] - ETAN[0, :, :]) / 86400

if True:                                 # we are using this
    tmp1 = mk3D_mod(mygrid['DRF'],hfC) * hfC
    tmp2 = tmp1/mk3D_mod(DD,tmp1)
    tmptend = tmp2 * mk3D_mod(ETAN, tmp2) * 1029 * mk3D_mod(RAC, hfC)

In [17]:
isinstance(tmp2, np.ndarray)
half_size = np.array(ETAN.shape)
len(half_size) == 2

True

In [19]:
arr_lessd = mygrid['DRF']
arr_3d = hfC

tmp1 = arr_3d.copy()
tmp2 = tmp1.shape
n1 = tmp2[2]
n2 = tmp2[1]
tmp1 = np.dot(np.ones((n1*n2,1)),arr_lessd[np.newaxis,:])
tmp1 = np.reshape(tmp1,(arr_3d.shape[0],n2,n1))
tmp1.shape

(23, 16, 20)